# Meal Classification using Continuous Glucose Monitors
## Purpose
In this project, you will use a training dataset to train and test a machine model. The purpose is to distinguish between meal and no meal time series data.


## Project Description
In this project, we are considering the Artificial Pancreas medical control system, specifically the Medtronic 670G system. The Medtronic system consists of a continuous glucose monitor (CGM) and the Guardian Sensor (12), which is used to collect blood glucose measurements every 5 minutes. The sensor is single-use and can be used continuously for 7 days after which it has to be replaced. The replacement procedures include a recalibration process that requires the user to obtain blood glucose measurements using a Contour NextLink 2.4 glucosemeter ®.
 
1
Note that this process also requires manual intervention. The Guardian Link Transmitter® powers the CGM sensor and sends the data to the MiniMed 670G® insulin pump. The insulin pump utilizes the Smart Guard Technology that modulates the insulin delivery based on the CGM data. The SmartGuard Technology uses a Proportional, Integrative, and Derivative controller to derive small bursts of insulin also called Micro bolus to be delivered to the user. During meals, the user uses a BolusWizard to compute the amount of food bolus required to maintain blood glucose levels. The user manually estimates the amount of carbohydrate intake and enters it into the Bolus Wizard.
The Bolus Wizard is pre-configured with the correction factor, body weight, and average insulin sensitivity of the subject, and it calculates the bolus insulin to be delivered. The user can then program the MiniMed 670G infusion pump to deliver that amount. In addition to the bolus, the MiniMed 670G insulin pump can also provide a correction bolus. The correction bolus amount is provided only if the CGM reading is above a threshold (typically 120 mg/dL) and is a proportional amount with respect to the difference between the CGM reading and the threshold.
The SmartGuard technology has two methods of suspending insulin delivery: a) Suspend on low, where the insulin delivery is stopped when the CGM reading is less than a certain threshold, or b) suspend on predicted low, where the insulin delivery is stopped when the CGM reading is predicted to be less than a certain threshold. Apart from these options, insulin delivery can also be suspended manually by the user or can be suspended when the insulin reservoir is running low.

## Extraction: Meal data
1. From the InsulinData.csv, search column Y(BWZ Carb Input(grams)) for a non-NAN non-zero value. This time indicates the start of the meal consumption time tm.
2. FromCGMData.csv,get the glucose time series data corresponding to themeal(tm).
3. Mealdata comprises a 2hr30 min stretch of CGMData. So once you get the corresponding glucose time look for a 2hr stretch (tm+2hrs) which is the postprandial period. In addition, take a 30 min stretch (tm-30min) before the start of the meal (tm).
4. To extract the meal data there can be three conditions:
    - There is no meal from time tm to time tm+2hrs.Then use this stretch as meal data
    - There is a meal at some time tp in between tp>tm and tp<tm+2hrs.Ignore the meal data at time tm and consider the meal at time tp instead. For example, suppose the start of the meal (tm) is at 9:15 and you find a meal at 10:00 (tp) where tp > tm and tp < tm+2hrs, consider the meal at tp.
    - There is a meal at time tm+2hrs, then consider the stretch from tm+1hr 30min to tm+4hrs as meal data.
5. Eachstretchwillbeonerowandwillhave30columnsi.e.2hr30minstretchforevery5 minutes in CGMData.csv. After extracting the all meal data, you need to create a Meal Data Matrix (P x 30) where P is the total number of rows for the meal data time series.


## Feature Extracion

1. Time difference between the time meal was taken and the time the CGM levels get the highest peak
2. CGM difference between the CGM at the meal time and the highest CGM level after having a meal
3. Fast Fourier Transform: frequency1(f1)
4. Fast Fourier Transform: f(f1)
5. Fast Fourier Transform: frequency2(f2)
6. Fast Fourier Transform: f(f2)
7. Fast Fourier Transform: derivative of f1
8. Fast Fourier Transform: derivative of f2

## Functions
The project required to process multiple datasets, therefore an implementation of multiple function to encapsulate the entire cleaning, meal/no-meal extraction and feature extracion process was necessary. 

In [23]:
# Function will return
def get_peak_values(row: pd.Series) -> pd.Series:
    # reset the index in case to start a 0
    count = len(row)
    row.index = range(count)
    # get all the peaks indexes
    peak_idx, _ = scipy.signal.find_peaks(row)
    # Return peak indexes and values
    return row[peak_idx]


def get_highest_peak_idx(row: pd.Series) -> int:
    # Get peak values and reset index to start from 0
    peaks = get_peak_values(row)
    # if no peaks were found, retun 0
    if len(peaks) == 0:
        return 0
    return peaks.idxmax()


# will compute the time difference between the time meal was taken
# and the time the CGM levels get the highest peak
# Function will take a list() of 24 items where each item
# represent a 5 minutes interval
# return the time difference in minutes
def get_time_to_reach_high_peak(row: pd.Series) -> int:
    # Get the index at highest peak
    highest_peak_idx = get_highest_peak_idx(row)
    # # if no peak were found, return Nan
    # if highest_peak_idx == -1:
    #     return np.nan
    # total time in minutes = highest peak index x 5
    return highest_peak_idx * 5


# Calculate the CGM difference between the CGM at the meal time
# and the highest CGM level after having a meal
def get_cgm_difference(row: pd.Series) -> int:
    # Get the index at highest peak
    highest_peak_idx = get_highest_peak_idx(row)
    # if highest_peak_idx == -1:
    #     return np.nan
    # Return CGM difference
    return row[highest_peak_idx] - row[0]


def get_fft_power_frequency(row: pd.Series, peaks: int = 2) -> pd.Series:
    fft = pd.Series(abs(np.fft.fft(row)))
    fft_peaks = get_peak_values(fft)
    peak_count = len(fft_peaks)

    if peak_count < peaks:
        not_found = peaks - peak_count
        return pd.Series(list(sum(fft_peaks.to_dict().items(), ())) + [0] * 2 * not_found)
        # Return a pandas Series
    # [frequency_1, P(frequency_1), ..., frequency_#peaks, P(frequency#peaks)]
    return pd.Series(sum(fft_peaks[:peaks].to_dict().items(), ()))


# Get dcgm/dt for 24 input row
def get_diff1_cgm(row: pd.Series) -> int:
    highest_peak_idx = get_highest_peak_idx(row)
    diff_cgm_values = np.diff(row)
    for i in range(highest_peak_idx - 1, -1, -1):
        if diff_cgm_values[i] > 0:
            return diff_cgm_values[i]
    return 0


# Get d2cgm/dt for 24 input row
def get_diff2_cgm_at_tm(row: pd.Series) -> int:
    # Get dcgm/dt and d2cgm/dt
    d1_cgm = np.diff(row)
    d2_cgm = np.diff(d1_cgm)
    # Return the highest value closest to tm
    return max(d2_cgm[:4])


def feature_extractor(meal_data: pd.DataFrame) -> pd.DataFrame:
    tm_idx = 0
    column_size = meal_data.shape[1]
    if column_size == 30:
        tm_idx = 6
    columns = range(tm_idx, column_size)
    meal_feature_df = pd.DataFrame(meal_data.iloc[:, -24:].apply(get_time_to_reach_high_peak, axis=1),
                                   columns=['Time to Reach Max Peak'])
    meal_feature_df['CGM difference Normalized'] = meal_data.iloc[:, -24:].apply(get_cgm_difference, axis=1)
    # Normalize the difference. (difference / initial value)
    meal_feature_df['CGM difference Normalized'] = meal_feature_df['CGM difference Normalized'] / meal_data.iloc[:, -24]
    # Compute the frequency 1, the Power of f1, the frequency 2, the Power of f2
    # using Fast Fourier Transform
    meal_feature_df[['f1', 'P_f1', 'f2', 'P_f2']] = meal_data.iloc[:, -24:].apply(get_fft_power_frequency, axis=1)
    meal_feature_df['(d1/dt)(CGM)'] = meal_data.iloc[:, -24:].apply(get_diff1_cgm, axis=1)
    meal_feature_df['(d2/dt)(CGM)'] = meal_data.iloc[:, -24:].apply(get_diff2_cgm_at_tm, axis=1)

In [24]:
# Function will read the csv files and clean the data and extract 30 values and 24 values
# for each instance of meal and no meal, respectively.
# it takes the path to cgm and insulin CVS files and
# return a list with meal and no meal DataFrame
def get_meal_data(cgm_file: str, insulin_file: str, index_col: list = [None, None]) -> list[pd.DataFrame]:
    # Generate all possible no meal start time based on the time between meals
    # and return them in a list
    def get_no_meal_tm(row):
        start_dates = list()
        if not pd.isnull(row['Time Between Meals']):
            # Get the total number of 2-hours valid intervals between meals
            total_hours = row['Time Between Meals'] // pd.Timedelta(hours=2)
            # start the range at 1 to skip the first 2-hours interval
            # since it belongs to meal data
            for hours in range(1, int(total_hours)):
                tm = row['Meal Consumption Time'] + pd.Timedelta(hours=hours * 2)
                start_dates.append(tm)
        return start_dates

    # Load the dataset from the Continuous Glucose Sensor into CGM_df DataFrame
    CGM_df = pd.read_csv(cgm_file, index_col=index_col[0])
    # Reduce the dimensionality of the dataset by removing the irrelevant columns
    # containing information that is not useful for this project
    CGM_df = CGM_df[['Date', 'Time', 'Sensor Glucose (mg/dL)']]
    # Rename Column 'Sensor Glucose (mg/dL)' to 'CGM' to match
    # the variable name specified for this project
    CGM_df.rename(columns={'Sensor Glucose (mg/dL)': 'CGM'}, inplace=True)
    # Generate a valid datetime column using 'Date' and 'Time' columns
    # if a valid datetime can't be generated, it will be set as Nan
    CGM_df['Datetime'] = pd.to_datetime(CGM_df['Date'] + ' ' + CGM_df['Time'], errors='coerce')
    # sort the DataFrame by datetime
    CGM_df.sort_values(by=['Datetime'], inplace=True)
    # Set the sorted datetime column as the index
    CGM_df.set_index('Datetime', inplace=True)

    # Load the start of meal consumption time data from the insulinData.csv into insulin_df DataFrame
    insulin_df = pd.read_csv(insulin_file, index_col=index_col[1])
    # Reduce the dimensionality of the dataset by removing the irrelevant columns
    # containing information that is not useful for this project
    insulin_df = insulin_df[['Date', 'Time', 'BWZ Carb Input (grams)']]

    # Retrieve only the instances with meal amount value greater than 0.
    insulin_df = insulin_df[insulin_df['BWZ Carb Input (grams)'] > 0]

    # Generate a valid datetime column using 'Date' and 'Time' columns
    # if a valid datetime can't be generated, it will be set as Nan
    insulin_df['Meal Consumption Time'] = pd.to_datetime(insulin_df['Date'] + ' ' + insulin_df['Time'], errors='coerce')
    # sort the DataFrame by datetime
    insulin_df.sort_values(by='Meal Consumption Time', inplace=True)
    # Add a new column called 'Next Meal Consumption Time'. This new column and combination
    # with the column 'Meal Consumption Time' will be used to calculate the time between meals
    insulin_df['Next Meal Consumption Time'] = insulin_df['Meal Consumption Time'].shift(-1)
    insulin_df['Time Between Meals'] = insulin_df['Next Meal Consumption Time'] - insulin_df['Meal Consumption Time']

    insulin_df['No Meal Start dates'] = insulin_df.apply(get_no_meal_tm, axis=1)

    # Retrieve all the no meals start date and time
    valid_no_meal_dates = insulin_df['No Meal Start dates'].tolist()
    # Flatten the list of lists
    valid_no_meal_dates = sum(valid_no_meal_dates, [])

    no_meal_data = list()
    no_meal_threshold = 5
    for valid_no_meal_date in valid_no_meal_dates:
        tm_index = CGM_df.index.get_indexer([valid_no_meal_date], method='nearest', tolerance=pd.Timedelta('5min'))[0]
        if tm_index > -1:
            tm = CGM_df.index[tm_index]
            end = tm + pd.Timedelta(hours=2)
            no_meal_instance = CGM_df.loc[tm:end]['CGM']

            if len(no_meal_instance) >= 24 and no_meal_instance.isna().sum() < no_meal_threshold:
                no_meal_cgm_values = no_meal_instance.interpolate().values
                no_meal_data.append(no_meal_cgm_values[:24])

    no_meal_df = pd.DataFrame(no_meal_data)
    no_meal_df.dropna(inplace=True)

    # Obtain the valid starting date and times to extract meal data.
    valid_meal_dates = insulin_df[insulin_df['Time Between Meals'] >= '02:00:00']['Meal Consumption Time'].values

    # Find the CGM_df dataframe indexes with the closest date and time for meal consumption found in insulin_df
    # Compare the valid meal dates found in insulin_df DataFrame to the CGM_df dataframe Datetime indexes
    # and find the indexes with the closest date and time for meal consumption in CGM_df
    meal_indexes = CGM_df.index.get_indexer(valid_meal_dates, method='nearest', tolerance=pd.Timedelta('5m'))
    # get_indexer() function will return -1 for dates not found within the set
    # tolerance. Thus, -1 values will be removed
    meal_indexes = meal_indexes[meal_indexes > -1]

    meal_data = list()
    meal_threshold = 2
    for valid_meal_date in valid_meal_dates:
        # Compare the valid meal dates found in insulin_df DataFrame to the CGM_df dataframe Datetime indexes
        # and find the indexes with the closest date and time for meal consumption in CGM_df
        tm_index = CGM_df.index.get_indexer([valid_meal_date], method='nearest', tolerance=pd.Timedelta('5m'))[0]
        #
        if tm_index > -1:
            # Set the 2.5 hours stretch of CGM Data, where tm represent the start of each meal.
            # start is 30 minutes before  tm and end 2 hours after tm
            tm = CGM_df.index[tm_index]
            start = tm - pd.Timedelta(minutes=30)
            end = tm + pd.Timedelta(hours=2)
            # Retrieve all the instances between the 2.5 hours stretch
            meal_instance = CGM_df.loc[start:end]['CGM']

            if len(meal_instance) >= 30 and meal_instance.isna().sum() < meal_threshold:
                meal_cgm_values = meal_instance.interpolate().values
                meal_data.append(meal_cgm_values[:30])

    meal_df = pd.DataFrame(meal_data)
    meal_df.dropna(inplace=True)

    return [meal_df, no_meal_df]

In [31]:
import pandas as pd
import numpy as np
import scipy
import warnings
import pickle
from sklearn.svm import SVC
from scipy import signal
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import datetime as dt
warnings.filterwarnings("ignore")



# Get the meal and no meal data for first cgm file and patient 2 data 
meal_df, no_meal_df = get_meal_data('CGMData.csv', 'InsulinData.csv')
pt2_meal_df, pt2_no_meal_df = get_meal_data('CGM_patient2.csv', 'Insulin_patient2.csv', ['Index', 'Index'])

# Extract the feature matrix for the every meal data
meal_features = feature_extractor(meal_df)
no_meal_features = feature_extractor(no_meal_df)

pt2_meal_features = feature_extractor(pt2_meal_df)
pt2_no_meal_features = feature_extractor(pt2_no_meal_df)

# Assign their corresponding class value
meal_features['Class'] = 1
no_meal_features['Class'] = 0

pt2_meal_features['Class'] = 1
pt2_no_meal_features['Class'] = 0


In [52]:
meal_training_data = pd.concat([meal_features, no_meal_features, pt2_no_meal_features], axis=0)

# select all training features
X = meal_training_data.iloc[:, :-1]
# Select label vector
y = meal_training_data['Class']

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Note: select the classifier with better score
dt_clf = DecisionTreeClassifier(max_depth=4, min_samples_leaf=10, random_state=0)
dt_clf.fit(X, y)
y_pred = dt_clf.predict(X_test)

print('dt accuracy_score score: ', accuracy_score(y_test, y_pred))


dt accuracy_score score:  0.845841784989858
